In [1]:
import random
from pprint import pprint
import io
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import numpy as np
import time
import logging

import tyssue
from tyssue import History
from tyssue.dynamics import effectors, model_factory, model_factory_vessel, model_factory_cylinder
from tyssue.generation.hexagonal_grids import hexa_cylinder
from tyssue.generation.shapes import sheet_from_cell_centers
from tyssue.geometry.sheet_geometry import CylinderGeometry, CylinderGeometryInit
from tyssue import config
from tyssue.draw import sheet_view, create_gif
from tyssue.geometry.vessel_geometry import VesselGeometry as geom
from tyssue.solvers.viscous import EulerSolver
from tyssue.draw.ipv_draw import browse_history
from tyssue.topology.sheet_topology import cell_division, drop_face, split_vert
from tyssue.topology.base_topology import remove_face
from tyssue.behaviors import EventManager
from tyssue.generation.modifiers import create_anchors
from vivarium_tyssue.behaviors import *
import ipyvolume as ipv

from process_bigraph import Process, Composite, ProcessTypes
from process_bigraph.emitter import emitter_from_wires, gather_emitter_results

%load_ext snakeviz

C++ extensions are not available for this version
collision solver could not be imported You may need to install CGAL and re-install tyssue


In [2]:
# def fix_points_cylinder(sheet, radius):
#     xy = sheet.vert_df[['x', 'y']].to_numpy()
#     r = np.linalg.norm(xy, axis=1)
#     r_safe = np.where(r == 0, 1e-12, r)
#     xy_on_cylinder = (radius/r_safe)[:, None] * xy
#     sheet.vert_df['x'] = xy_on_cylinder[:, 0]
#     sheet.vert_df['y'] = xy_on_cylinder[:, 1]

In [3]:
# def divide_cylinder(sheet, manager, geom, division_rate, dt, radius):
#     update_stem_cells(sheet)
#     stem_cells = sheet.face_df.loc[(sheet.face_df["stem_cell"] == 1) & (sheet.face_df["area"]>=0.7*sheet.face_df["area"].mean())].copy()
#     n_stem = len(stem_cells)
#     cell_ids = list(stem_cells["unique_id"])
#     n_divisions = np.random.binomial(n=n_stem, p=division_rate * dt)
#     dividing_cells = np.random.choice(cell_ids, size=n_divisions, replace=False)
#
#     for cell_id in dividing_cells:
#         cell_idx = int(sheet.face_df[sheet.face_df["unique_id"] == cell_id].index[0])
#         daughter = cell_division(sheet, cell_idx, geom)
#     manager.append(divide_cylinder, geom=geom, division_rate=division_rate, dt=dt, radius=radius)
#     fix_points_cylinder(sheet, radius=radius)

In [4]:
# def apoptosis_cylinder(sheet, manager, death_rate, dt, radius, geom):
#     update_stem_cells(sheet)
#     dying_cells = sheet.face_df.loc[sheet.face_df["dying_cell"] == 1]
#     n_dying = len(dying_cells)
#     cell_ids = list(dying_cells["unique_id"])
#     n_deaths = np.random.binomial(n=n_dying, p=death_rate * dt)
#     to_kill = np.random.choice(cell_ids, size=n_deaths, replace=False)
#
#     for cell_id in to_kill:
#         cell_idx = int(sheet.face_df[sheet.face_df["unique_id"] == cell_id].index[0])
#         # if sheet.face_df.loc[cell_idx, "boundary"] == 0:
#         vertex = remove_face(sheet, cell_idx)
#         # else:
#         #     vertex = drop_face(sheet, cell_idx, geom)
#         # split_vert(sheet, vertex)
#     manager.append(apoptosis_cylinder, death_rate=death_rate, dt=dt, radius=radius, geom=geom)
#     fix_points_cylinder(sheet, radius=radius)

In [5]:
def auto_correlated_tension(sheet, manager, sigma, phi):
    """
    Parameters:
    sheet: tyssue.Sheet object
    manager: EventManager object
    sigma: float, standard deviation
    phi: float, autocorrelation factor between 0 and 1 (AR(1) autocorrelation factor)
        closer to 1 means stronger correlation
    Returns:
    updates sheet.edge_df["line_tension"]
    """
    noise = np.random.normal(loc=0, scale=sigma, size=len(sheet.edge_df["line_tension"]))
    sheet.edge_df["line_tension"] =  phi * sheet.edge_df["line_tension"] + noise
    manager.append(auto_correlated_tension, sigma=sigma, phi=phi)

In [6]:
# def update_stem_cells(eptm):
#     eptm.face_df['stem_cell'] = 0
#     eptm.face_df['dying_cell'] = 0
#     eptm.face_df.loc[(eptm.face_df["boundary"] != 1) & (eptm.face_df["z"] < 0), "stem_cell"] = 1
#     eptm.face_df.loc[(eptm.face_df["z"] > 0), "dying_cell"] = 1

In [7]:
radius = 2.576
#generate cylindrical tissue
points_xyz = hexa_cylinder(16,30, radius = radius, noise = 0.0, capped = True)
sheet = sheet_from_cell_centers(points_xyz)
sheet = sheet.extract_bounding_box(z_boundary = (-10.1, 30), coords=['x', 'y', 'z'])
sheet.sanitize(trim_borders=False)
create_anchors(sheet)
sheet = sheet.extract_bounding_box(z_boundary = (-10.1, 10), coords=['x', 'y', 'z'])
sheet.sanitize(trim_borders=False)
geom.update_all(sheet)
sheet.network_changed = False

/Users/tasnifrahman/PycharmProjects/vivatyssue/src/tyssue/generation/modifiers.py:193: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet.edge_df = pd.concat([sheet.edge_df, anchor_edge_df], sort=True)


In [8]:
sheet.face_df.loc[(sheet.face_df["z"]<8) & (sheet.face_df["z"]>-8), "area"].mean()

np.float64(1.0001577509936472)

In [9]:
draw_specs = tyssue.config.draw.sheet_spec()
draw_specs["face"]["visible"] = True
draw_specs["face"]["visibles"] = True
draw_specs["face"]["alpha"] = 0.2
ipv.clear()
fig, mesh = sheet_view(sheet, coords=['x', 'y', 'z'], mode="3D", **draw_specs)
ipv.pylab.xyzlabel("x-axis", "y-axis", "z-axis")
ipv.pylab.style.axes_on()
fig

Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.0, position=(0.0,…

In [10]:
#generate model
model = model_factory_vessel([
    effectors.LineTension,
    effectors.FaceAreaElasticity,
    effectors.PerimeterElasticity,
], effectors.FaceAreaElasticity)

In [11]:
#set model parameters
sheet.face_df["area_elasticity"] = 1
sheet.face_df["prefered_area"] = 1
sheet.face_df["perimeter_elasticity"] = 1
sheet.face_df["prefered_perimeter"] = 3.5
sheet.edge_df["line_tension"] = np.random.normal(loc=0, scale=0.01, size=len(sheet.edge_df))
sheet.edge_df["is_active"] = 1
sheet.vert_df["viscosity"] = 5
sheet.vert_df["prefered_deviation"] = 0
sheet.vert_df["surface_elasticity"] = 5
sheet.settings["threshold_length"] = 0.1
geom.update_all(sheet)

In [12]:
sheet.edge_df["face"]

edge
0       123
1       123
2       123
3       123
4       123
       ... 
1915    284
1916    284
1917    284
1918    284
1919     68
Name: face, Length: 1920, dtype: int64

In [13]:
history = History(sheet)

In [14]:
#set time-step
dt=0.1

In [15]:
#initialize manager and add division and apoptosis
#set division and apoptosis parameters
manager = EventManager()
manager.append(divide_cylinder, geom=geom, division_rate=0.0005, dt=dt, radius=radius)
manager.append(apoptosis_cylinder, death_rate=0.0005, dt=dt, radius=radius, geom=geom)
manager.append(auto_correlated_tension, sigma = 0.01, phi=0.5)

In [16]:
#initialize solver
solver = EulerSolver(
    sheet,
    geom,
    model,
    history=history,
    auto_reconnect=True,
    manager=manager,
)

In [17]:
sheet.edge_df["is_active"]

edge
0       1
1       1
2       1
3       1
4       1
       ..
1915    1
1916    1
1917    1
1918    1
1919    1
Name: is_active, Length: 1920, dtype: int64

In [ ]:
#set time variables
dt = dt
tf = 2000
sheet.settings["dt"] = dt
sheet.settings["p_4"] = 1/(dt)
sheet.settings["p_5p"] = 1/(dt)
#run simulation
start = time.time()
res = solver.solve(tf=tf, dt = dt)
print(f"Time Elapsed: {time.time()-start}")

In [19]:
sheet.edge_df

,at_border,boundary,cell,cell_o,cx,cy,cz,dx,dy,dz,...,trgt,trgt_o,tx,ty,tz,unique_id,unique_id_max,ux,uy,uz
edge,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0.0,0.0,0.0,-0.008241,-0.010771,0.618845,...,26,410,-2.049970,1.559936,6.772272,0,6120,-0.013314,-0.017401,0.999759
1,0,0,0,0,0.0,0.0,0.0,0.017060,-0.285383,-0.243086,...,424,844,-2.558917,-0.296174,-6.927873,10,6120,0.045460,-0.760445,-0.647738
2,0,0,0,0,0.0,0.0,0.0,-0.197723,0.760734,-0.314483,...,130,842,-2.562840,-0.260048,-6.369028,12,6120,-0.233519,0.898456,-0.371417
3,0,0,0,0,0.0,0.0,0.0,-0.064084,0.132503,0.831375,...,423,841,-2.350115,-1.054862,-6.934672,14,6120,-0.075901,0.156936,0.984680
4,0,0,0,0,0.0,0.0,0.0,0.188915,-0.472931,0.228710,...,128,840,-2.286032,-1.187365,-7.766047,16,6120,0.338281,-0.846855,0.409540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767,0,0,0,0,0.0,0.0,0.0,-0.000173,0.000016,0.119858,...,604,449,0.244245,2.564395,6.105748,2242,6120,-0.001445,0.000138,0.999998
1768,0,0,0,0,0.0,0.0,0.0,-0.014753,0.055357,0.186684,...,614,839,-2.496348,-0.635628,-9.210861,865,6120,-0.075547,0.283480,0.955999
1769,0,0,0,0,0.0,0.0,0.0,0.014753,-0.055357,-0.186684,...,81,681,-2.481595,-0.690985,-9.397545,1905,6120,0.075547,-0.283480,-0.955999


In [20]:
ipv.clear()

In [21]:
ipv.clear()
browse_history(history,**draw_specs)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

interactive(children=(IntSlider(value=0, description='i', max=20000), Output()), _dom_classes=('widget-interac…

In [22]:
len(sheet.face_df)

299

In [23]:
sheet.edge_df["is_active"]

edge
0       1
1       1
2       1
3       1
4       1
       ..
1767    1
1768    1
1769    1
1770    1
1771    1
Name: is_active, Length: 1772, dtype: int64

In [24]:
ipv.clear()
fig, mesh = sheet_view(sheet, coords=['x', 'y', 'z'], mode="3D", **draw_specs)
ipv.pylab.xyzlabel("x-axis", "y-axis", "z-axis")
ipv.pylab.style.axes_on()
fig

Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.0, position=(0.0,…

In [25]:
sheet1 = history.retrieve(0.0)

In [26]:
sheet.edge_df

,at_border,boundary,cell,cell_o,cx,cy,cz,dx,dy,dz,...,trgt,trgt_o,tx,ty,tz,unique_id,unique_id_max,ux,uy,uz
edge,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0.0,0.0,0.0,-0.008241,-0.010771,0.618845,...,26,410,-2.049970,1.559936,6.772272,0,6120,-0.013314,-0.017401,0.999759
1,0,0,0,0,0.0,0.0,0.0,0.017060,-0.285383,-0.243086,...,424,844,-2.558917,-0.296174,-6.927873,10,6120,0.045460,-0.760445,-0.647738
2,0,0,0,0,0.0,0.0,0.0,-0.197723,0.760734,-0.314483,...,130,842,-2.562840,-0.260048,-6.369028,12,6120,-0.233519,0.898456,-0.371417
3,0,0,0,0,0.0,0.0,0.0,-0.064084,0.132503,0.831375,...,423,841,-2.350115,-1.054862,-6.934672,14,6120,-0.075901,0.156936,0.984680
4,0,0,0,0,0.0,0.0,0.0,0.188915,-0.472931,0.228710,...,128,840,-2.286032,-1.187365,-7.766047,16,6120,0.338281,-0.846855,0.409540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767,0,0,0,0,0.0,0.0,0.0,-0.000173,0.000016,0.119858,...,604,449,0.244245,2.564395,6.105748,2242,6120,-0.001445,0.000138,0.999998
1768,0,0,0,0,0.0,0.0,0.0,-0.014753,0.055357,0.186684,...,614,839,-2.496348,-0.635628,-9.210861,865,6120,-0.075547,0.283480,0.955999
1769,0,0,0,0,0.0,0.0,0.0,0.014753,-0.055357,-0.186684,...,81,681,-2.481595,-0.690985,-9.397545,1905,6120,0.075547,-0.283480,-0.955999


In [27]:
ipv.clear()
browse_history(history, **draw_specs)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

interactive(children=(IntSlider(value=0, description='i', max=20000), Output()), _dom_classes=('widget-interac…